In [8]:
import urllib.request
from bs4 import BeautifulSoup
quote_page = "https://news.google.com/news/search/section/q/facebook/facebook?hl=en&gl=US&ned=us"
page = urllib.request.urlopen(quote_page)
soup = BeautifulSoup(page, "html.parser")
print(soup.prettify())
#look for aria-level="2"

<!DOCTYPE doctype html>
<html dir="ltr" lang="en">
 <head>
  <base href="https://news.google.com/news/"/>
  <meta content="origin" name="referrer"/>
  <link href="https://news.google.com/search/section/q/facebook/facebook" rel="canonical"/>
  <meta content="initial-scale=1,minimum-scale=1,maximum-scale=1,user-scalable=no" name="viewport"/>
  <meta content="yes" name="mobile-web-app-capable"/>
  <meta content="yes" name="apple-mobile-web-app-capable"/>
  <meta content="Google News" name="application-name"/>
  <meta content="Google News" name="apple-mobile-web-app-title"/>
  <meta content="black" name="apple-mobile-web-app-status-bar-style"/>
  <meta content="no" name="msapplication-tap-highlight"/>
  <link href="//encrypted-tbn0.gstatic.com" rel="preconnect"/>
  <link href="//encrypted-tbn1.gstatic.com" rel="preconnect"/>
  <link href="//encrypted-tbn2.gstatic.com" rel="preconnect"/>
  <link href="//encrypted-tbn3.gstatic.com" rel="preconnect"/>
  <link href="//fonts.googleapis.com" rel